# Homework 10

## Problem 1. 30 POINTS
Create a bash shell script that will create a sqlite database of the 4 csv files in this directory. Make sure that you assing meaningful data types and constraints to your database.

## Problem 2. 30 POINTS.
Using python, answer the following questions:
    1. Provide code that creates a pandas dataframe that contains a patient-level dataset with:
        - LastName
        - FirstName
        - Total Billing Amount
    2. The billing department needs to modify each billing statment so that we take the total billing amount for each person, multiply it by 4.2 and subtract 10. Create a new column on your pandas dataframe that applies this correction and name this variable `correction`.
    3. For all patients less than 40 years old, create a pandas data frame that contains:
        - LastName
        - FirstName
        - physician last name
        - physician first name
        - total times seen that physician
    

In [5]:
#Question 1
from sqlalchemy import create_engine
import pandas as pd

engine=create_engine('sqlite:///mydatabase.sqlite')


df=pd.read_sql('''select p.lastname,p.firstname,sum(e.billamount) as totalbill
from encounter e left join patient p on e.patientid==p.patientid
group by e.patientid;
''',engine)
df

,lastname,firstname,totalbill
0,doe,john,186.97
1,sickly,joseph,888.10
2,daily,bob,375.69


In [6]:
#Question 2
df['correction']=df['totalbill']*4.2-10
df

,lastname,firstname,totalbill,correction
0,doe,john,186.97,775.274
1,sickly,joseph,888.10,3720.020
2,daily,bob,375.69,1567.898


In [7]:
#Question 3
#extract essential information for processing
df3=pd.read_sql('''select p.lastname as patientlastname,
p.firstname as patientfirstname,phy.lastname as physicianlastname,
phy.firstname as physicianfirstname,
p.birthday
from encounter e 
left join patient p on e.patientid==p.patientid
left join physician phy on e.physician==phy.physicianid;
''',engine)
df3


,patientlastname,patientfirstname,physicianlastname,physicianfirstname,birthday
0,doe,john,whitecoat,bill,2/3/63
1,sickly,joseph,wei,liu,4/2/81
2,sickly,joseph,wei,liu,4/2/81
3,sickly,joseph,na,li,4/2/81
4,sickly,joseph,na,li,4/2/81
5,daily,bob,na,li,9/2/91
6,daily,bob,satterwhite,jill,9/2/91
7,sickly,joseph,na,li,4/2/81
8,doe,john,satterwhite,jill,2/3/63
9,sickly,joseph,whitecoat,bill,4/2/81


In [8]:
#add age column
df3.birthday=pd.to_datetime(df3.birthday)
df3.birthday[0]=pd.to_datetime("1963-02-03")
df3.birthday[8]=pd.to_datetime("1963-02-03")
df3['age']=2018-df3.birthday.dt.year



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [9]:
df3[df3.age<40]
df3test=df3.groupby(['patientlastname','patientfirstname','physicianlastname','physicianfirstname']).count()
df3test=df3test.drop(columns=['age'])
df3test=df3test.rename(dict(birthday='timesseenbyphysician'),axis=1)
df3test


timesseenbyphysician
patientlastname patientfirstname physicianlastname physicianfirstname                      
daily           bob              na                li                                     1
                                 satterwhite       jill                                   1
doe             john             satterwhite       jill                                   1
                                 whitecoat         bill                                   1
sickly          joseph           na                li                                     3
                                 wei               liu                                    2
                                 whitecoat         bill                                   1

## Problem 1. 15 POINTS
You should have received an email that contains:
    - host
    - user
    - password
    
Go ahead and create three environment variables that hold this information. From this point forward, if you need this information, you should refer to the environment variables and NOT the values themselves. You will lose all points, if the values themselves are in the code and submitted to github. You will have effectively disclosed very sensitive data. **NOTE**, there is no code you turn in to get points for this problem. Rather, I will know that you have done this by reviewing the code below.

## Problem 2. 5 POINTS
Create a `sqlalchemy` engine that is connected to our database. **NOTE** use the environment variables set up in problem 1 to do this. There should be NO host, username, or passwords visible if you are using environment variables correctly. You may receive some `ImportError`. You will need to figure out how to resolve these issues.

In [2]:
from sqlalchemy import create_engine
import os

string='postgresql://' + os.getenv('user') + ':' + os.getenv('password') + '@' + os.getenv('host')
enginefinal=create_engine(string)

## Problem 3. 20 POINTS
Using the engine created in Problem 2. Use sqlalchemy to help you create a hand drawn ER diagram. Please find a way to get you hand-drawn ER diagram into this repository. HINT, if you take a picture with your phone, you can use that image (which is just a file) to put into this directory.

In [10]:
from sqlalchemy.schema import MetaData
m=MetaData()
m.reflect(enginefinal)

for table in m.tables.values():
    print('************')
    print('TABLE: ' + table.name)
    print('COLUMNS:')
    for column in table.c:
        print('\t'+column.name)

************
TABLE: patient
COLUMNS:
	patientid
	lastname
	firstname
	gender
	dob
************
TABLE: physician
COLUMNS:
	physicianid
	lastname
	firstname
	specialty
************
TABLE: encounter
COLUMNS:
	encounterid
	patientid
	encounterdate
	encounterphysician
	encounterreason
	billingamount
************
TABLE: billing
COLUMNS:
	billingid
	encounterid
	code


In [11]:
tablepatient=pd.read_sql('select * from patient;',enginefinal)
print(tablepatient.dtypes)
tablephysician=pd.read_sql('select * from physician;',enginefinal)
print(tablephysician.dtypes)
tableencounter=pd.read_sql('select * from encounter;',enginefinal)
print(tableencounter.dtypes)
tablebilling=pd.read_sql('select * from billing;',enginefinal)
print(tablebilling.dtypes)

patientid     int64
lastname     object
firstname    object
gender       object
dob          object
dtype: object
physicianid     int64
lastname       object
firstname      object
specialty      object
dtype: object
encounterid             int64
patientid               int64
encounterdate          object
encounterphysician      int64
encounterreason        object
billingamount         float64
dtype: object
billingid       int64
encounterid     int64
code           object
dtype: object


In [14]:
m.reflect(bind=engine)
print(m.tables)

immutabledict({'patient': Table('patient', MetaData(bind=None), Column('patientid', INTEGER(), table=<patient>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x114290cc0>, for_update=False)), Column('lastname', VARCHAR(length=50), table=<patient>), Column('firstname', VARCHAR(length=50), table=<patient>), Column('gender', VARCHAR(length=1), table=<patient>), Column('dob', DATE(), table=<patient>), schema=None), 'physician': Table('physician', MetaData(bind=None), Column('physicianid', INTEGER(), table=<physician>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x1142a5fd0>, for_update=False)), Column('lastname', VARCHAR(length=50), table=<physician>), Column('firstname', VARCHAR(length=50), table=<physician>), Column('specialty', VARCHAR(length=50), table=<physician>), schema=None), 'encounter': Table('encounter', MetaData(bind=None), Column('encounterid', INTEGE

In [12]:
from sqlalchemy import inspect
insp = inspect(engine)


In [37]:
print(insp.get_primary_keys('patient'))
print(insp.get_foreign_keys('patient'))
print(insp.get_primary_keys('physician'))
print(insp.get_foreign_keys('physician'))
print(insp.get_primary_keys('encounter'))
print(insp.get_foreign_keys('encounter'))
print(insp.get_primary_keys('billing'))
print(insp.get_foreign_keys('billing'))

['patientid']
[]
['physicianid']
[]
['encounterid']
[{'name': None, 'constrained_columns': ['patientid'], 'referred_schema': None, 'referred_table': 'patient', 'referred_columns': ['patientid'], 'options': {}}]
['billingid']
[{'name': None, 'constrained_columns': ['encounterid'], 'referred_schema': None, 'referred_table': 'encounter', 'referred_columns': ['encounterid'], 'options': {}}]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SADeprecationWarning: Call to deprecated method get_primary_keys.  Use get_pk_constraint instead.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SADeprecationWarning: Call to deprecated method get_primary_keys.  Use get_pk_constraint instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SADeprecationWarning: Call to deprecated method get_primary_keys.  Use get_pk_constraint instead.
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SADeprecationWarning: Call to deprecated method get_primary_keys.  Use get_pk_constraint instead.
  import sys


![title](./IMG_3005.jpg)